In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import date, timedelta, datetime
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [ ]:
date_time_str = '2020-12-12'
today = datetime.strptime(date_time_str, '%Y-%m-%d')

end = today
start = end - timedelta(days = 6)
print(f'start:{start} and end:{end}')

In [ ]:
# latest = requests.get('https://pselookup.vrymel.com/api/stocks').json()["stocks"]
# ticker_codes = [x['ticker_symbol'] for x in latest]
# print(ticker_codes)

# with open("ticker_codes.csv", "w") as f:
#     f.write(','.join(ticker_codes))

ticker_codes = []
with open("ticker_codes.csv", "r") as f:
    ticker_codes = f.readline().split(',') 
#     
print(ticker_codes)

In [ ]:
#reference https://www.stockswechart.com/pse-chart/blue-chip-stocks-pse/
ticker_codes = [ "MFC", "SMPH", "SM", "SLF", "ALI", "FB", "AC", "BDO", "JGS", "MER", "SMC", "BPI",
                "JFC", "URC", "AEV", "MBT", "ICT", "AP", "GLO", "TEL", "HVN", "MEG", "GTCAP", "LTG", 'CEB',
                "AGI", "DMC", "MPI", "PGOLD", "BLOOM", "FDC", "MAC", 'LOTO', 'FLI', 'MM', 'DITO', 'PHR', 'ACEN' ]
# ticker_codes = ['BDO']
print(ticker_codes)

In [ ]:

quad = []
trip = []
doub = []
single = []
below = []
increased_symbols = {
    "Quadrupled": quad,
    "Tripled": trip,
    "Doubled": doub,
    "Same": single,
    "Below": below,
    "Status": "Not Started"
}
    
# end_datetime = datetime.now()
# start_datetime = end_datetime - timedelta(days=10)


start_datetime = '2020-11-27'

end_datetime = '2020-12-18'



print(f'start: {start_datetime} and end: {end_datetime}')

for code in ticker_codes:
    code = code.upper()
    # Get updated value from phisix-api
    try:
        print(f'looking up for {code}...')
    #     latest = requests.get(f'http://phisix-api.appspot.com/stocks/{code}.json', timeout=10).json()
    #     last_price = latest['stock'][0]['price']['amount']
    #     date = latest['as_of']
        my_url = f'https://pselookup.vrymel.com/api/stocks/{code}/history/{start_datetime}/{end_datetime}'
        print(my_url)
        #get previos data from pselookup
        data = requests.get(my_url, timeout=15).json()['history']
        result = pd.json_normalize(data)
        previous_averaged_volume = int(result['volume'].iloc[0:9].mean())
        todays_volume = result.volume.iloc[-1]
        todays_close = result.close.iloc[-1]
        todays_date = result.trading_date.iloc[-1]
        print(f'current:{todays_volume}, previous:{previous_averaged_volume}')

        if 4 <= (todays_volume / previous_averaged_volume):
            quad.append({'symbol': code,
                         'date': todays_date,
                         'price': todays_close,
                         'size': f'{(todays_volume / previous_averaged_volume):.2f}'})
        elif 3 <= (todays_volume / previous_averaged_volume):
            trip.append({'symbol': code,
                         'date': todays_date,
                         'price': todays_close,
                         'size': f'{(todays_volume / previous_averaged_volume):.2f}'})
        elif 2 <= (todays_volume / previous_averaged_volume):
            doub.append({'symbol': code,
                         'date': todays_date,
                         'price': todays_close,
                         'size': f'{(todays_volume / previous_averaged_volume):.2f}'})
        elif 1 <= (todays_volume / previous_averaged_volume):
            single.append({'symbol': code,
                           'date': todays_date,
                         'price': todays_close,
                           'size': f'{(todays_volume / previous_averaged_volume):.2f}'})
        else:
            below.append({'symbol': code,
                          'date': todays_date,
                         'price': todays_close,
                          'size': f'{(todays_volume / previous_averaged_volume):.2f}'})
        
    except Exception as e:
        print(f'Failed for Code{code}, {data}, {e}')
        continue
            
print()
print('===== QUAD =====')
for x in quad:
    print(x)
print('===== TRIP =====')
for x in trip:
    print(x)
print('===== DOUBLED =====')
for x in doub:
    print(x)
print('===== SINGLED =====')
print(f'total: {len(single)}')
print('===== LOW =====')
print(f'total: {len(below)}')


In [ ]:
print()
print('===== QUAD =====')